<a href="https://colab.research.google.com/github/KhizarAziz/RP/blob/master/RP_Open.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install wandb

In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import shutil
import os
import math
import dlib
from pathlib import Path
import wandb
from wandb.keras import WandbCallback
wandb.login()
#import fastai.vision

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + '0dev_folder/Data/'
colab_base_dir = '/content/'



> ## **`Github`**



In [ ]:
!git clone https://github.com/KhizarAziz/RP.git



> ## **`Download Datasets and Code`**



In [ ]:
#download imdb
# !wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar
#download WIKI
# !wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar

In [ ]:
#extract datasets
# !tar -xvf imdb_crop.tar -C /content/RP/dataset
# !tar -xvf wiki_crop.tar -C /content/RP/dataset



> ## **`Preprocess Datasets`**



In [ ]:
cd /content/RP/

In [ ]:
#Wiki-IDMB
# !python /content/RP/m_preprocess_dataset/pre_process_WIKI-IMDB.py
#Morph
#!python /content/RP/m_preprocess_dataset/pre_process_Morph.py
#FGNET
# !python /content/RP/m_preprocess_dataset/pre_process_FGNET.py



> ## **`Train`**



In [ ]:
cd /content/RP

In [ ]:
import math
from m_net_training import C3AE_net
from m_net_training import training_utils
import feather
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,TensorBoard,ReduceLROnPlateau
from keras.losses import kl_divergence,MAE
from keras.metrics import mae,Accuracy
import tensorflow as tf

In [ ]:
# initialize wandb with your project name and optionally with configutations.
category = 12
interval = 10
run = wandb.init(project='My-Project',
           config={
              "Age_Loss": 'mean_absolute_error',
              "CLR_METHOD": 'triangular',
              "MAX_LR": 0.01,
              "MIN_LR": 1.000e-7,
              "NUM_EPOCHS": 70,
              "STEP_SIZE": 7,
              "W1_Loss": 'kl_divergence',
              'architecture':'architecture??',
              'batch_size':64,
              'dataset':'wiki',
              'dropout':0.2,
              'img_ch':3,
              'img_h':64,
              'img_w':64,
              'random_enforcing':False,
              'random_erasing':False,
              'seed':2021,
              'test_split':0.2,
              'train_split':0.8,
              'use_SE_block':True,
              'use_white_norm':True
           })
config = wandb.config

In [ ]:
# Loading dataset (from .feather file)
dataset_dir = Path(f'/content/RP/dataset/{config.dataset}_crop')
dataset_df = pd.DataFrame(columns=["image_path","age", "gender", "image", "org_box", "landmarks"])
for fnames in dataset_dir.glob('*.feather'):
  df_chunk = feather.read_dataframe(dataset_dir.joinpath(fnames))
  dataset_df = pd.concat([dataset_df,df_chunk],ignore_index=True)
dataset_df = dataset_df.astype({'age': 'float64'})
dataset_df.shape

In [ ]:
trainset, testset = train_test_split(dataset_df, train_size=config.train_split, test_size=config.test_split, random_state=config.seed)

In [ ]:
input_shape = (config.img_h,config.img_w)
train_gen =     training_utils.img_and_age_data_generator(trainset,batch_size=config.batch_size,category=category,interval=interval,imgs_shape=input_shape,random_erasing=config.random_erasing,random_enforcing=config.random_enforcing,dropout=config.dropout)
validation_gen = training_utils.img_and_age_data_generator(testset,batch_size=config.batch_size,category=category,interval=interval,imgs_shape=input_shape,random_erasing=False,random_enforcing=False,dropout=0.0)

In [ ]:
index = 1
first_batch = next(iter(train_gen))
print(type(first_batch[1][0]),type(first_batch[1][1]))

In [ ]:
index += 1
Accepted_List = [first_batch[0][0][index],first_batch[0][1][index],first_batch[0][2][index]]
print(first_batch[1][0][index])
f,axarr = plt.subplots(nrows=1,ncols=3,figsize=(10,10))
for i in range(3):
    img = Accepted_List[i]
    axarr[i].imshow(img)

In [ ]:
age_distribution = [trainset["age"][(trainset.age >= x -10) & (trainset.age <= x)].count() for x in range(10, 101, 10)]
age_distribution = [age_distribution[0]] + age_distribution + [age_distribution[-1]]
print(age_distribution)

In [ ]:
import imp
imp.reload(C3AE_net)
tf.keras.backend.clear_session()
model = C3AE_net.build_model(Categories = category, using_SE=config.use_SE_block, using_white_norm=config.use_white_norm)

### **`Compile_model`**

In [ ]:
adam = Adam(lr=config.MIN_LR)
weight_factor = 10
model.compile(
    optimizer=adam,
    loss = {'W1':config.W1_Loss,'age':config.Age_Loss},
    metrics={"age": mae,"W1":'accuracy'},
    loss_weights={'W1':weight_factor, 'age': 1}
)

In [ ]:
class meriApniCustom_lr_history(tf.keras.callbacks.Callback):
  def on_epoch_begin(self,epoch, logs={}):
      lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
      print('our lr is: {} '.format(lr))

In [ ]:
save_path = '/content/RP/m_models_saved/morph-'
callbacks = [
  ModelCheckpoint(save_path+'weights.{epoch:02d}-{val_age_mean_absolute_error:.2f}.hdf5',
                  monitor='val_age_mean_absolute_error',
                  verbose = 1,
                  save_best_only=True,
                  model ='min'),
 
  training_utils.CyclicLR(
                    mode=config.CLR_METHOD,
                    base_lr=config.MIN_LR,
                    max_lr=config.MAX_LR,
                    step_size=config.STEP_SIZE * (trainset.shape[0] // config.batch_size)),
  meriApniCustom_lr_history(),
 
  WandbCallback(monitor='val_age_loss')
]

### **`Fit`**

In [ ]:
history = model.fit(train_gen, steps_per_epoch=len(trainset) / config.batch_size, epochs=config.NUM_EPOCHS, callbacks=callbacks, validation_data=validation_gen, validation_steps=len(testset) / config.batch_size * 3)



> ## **`Inference`**






In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/C3AE/detect/shape_predictor_68_face_landmarks.dat")
def gen_face(image):
  face_rect_list = detector(image)
  xmin, ymin, xmax, ymax = face_rect_list[0].left() , face_rect_list[0].top(), face_rect_list[0].right(), face_rect_list[0].bottom() # face_rect is dlib.rectangle object, so extracting values from it
  lmarks_list = dlib.full_object_detections()
  for face_rect in face_rect_list:
    lmarks_list.append(predictor(image, face_rect))
  return np.array([xmin, ymin, xmax, ymax]), lmarks_list

# algorithm is not finalized, as i am still in research phase.... but workable code

def gen_boundbox(box, landmark):
    xmin, ymin, xmax, ymax = box 
    w, h = xmax - xmin, ymax - ymin
    nose_x, nose_y = (landmark.parts()[30].x, landmark.parts()[30].y)
    w_h_margin = abs(w - h)
    top2nose = nose_y - ymin

    return np.array([
        [(xmin - w_h_margin, ymin - w_h_margin), (xmax + w_h_margin, ymax + w_h_margin)],  # out
        [(nose_x - top2nose, nose_y - top2nose), (nose_x + top2nose, nose_y + top2nose)],  # middle
        [(nose_x - w//2, nose_y - w//2), (nose_x + w//2, nose_y + w//2)]  # inner box
    ])

In [ ]:
img = cv2.imread('/content/test.jpg')
model = models

In [ ]:
try:
    bounds, lmarks = gen_face(img)
except Exception as e:
  print(e)
padding = 200
cv2.copyMakeBorder(img, padding, padding, padding, padding, cv2.BORDER_CONSTANT)

In [ ]:
for pidx,landmarks in enumerate(lmarks):
    trible_box = gen_boundbox(bounds, landmarks)
    tri_imgs = []
    for bbox in trible_box:
        bbox = bbox #+ padding
        h_min, w_min = bbox[0]
        h_max, w_max = bbox[1]
        tri_imgs.append(cv2.resize(new_bd_img[w_min:w_max, h_min:h_max], (64, 64)))
    print(np.array(tri_imgs).shape)

In [ ]:
testing_imgs = []
for img in tri_imgs:
  new_img = np.expand_dims(img,axis=0)
  testing_imgs.append(new_img)

In [ ]:
result = models.predict(testing_imgs)

In [ ]:
result

In [ ]:
img = tri_imgs[0]

In [ ]:
a = np.expand_dims(img,axis=0)